In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math
import warnings
warnings.filterwarnings("ignore")
import statsmodels.formula.api as smf

In [2]:
# colnames = ['day','order_id','item_det_info','pay_timestamp','buyer_id','promise_speed','if_cainiao','merchant_id','Logistics_review_score']
data = pd.read_csv("../replication/mic_bin.csv").drop(columns = ['Unnamed: 0','level_0']).rename(columns = {'fraction':'action_time','Days':'day_count'})
data.head()

,order_id,order_date,logistic_order_id,action,facility_id,facility_type,city_id,logistic_company_id,timestamp,day,...,facility_count,week,day_values,day_of_week,recieve_count,arrive_count,depart_count,scan_count,consign_count,post_action_time
0,100534527,20170115,53756568.0,CONSIGN,NaN,NaN,NaN,532.0,2017-01-16 08:49:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264
1,100534527,20170115,53756568.0,SIGNED,112694.0,NaN,NaN,532.0,2017-01-19 16:51:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264
2,100534527,20170115,53756568.0,GOT,NaN,NaN,NaN,532.0,2017-01-16 18:00:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264
3,100534527,20170115,53756568.0,DEPARTURE,NaN,NaN,NaN,532.0,2017-01-17 06:54:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264
4,100534527,20170115,53756568.0,ARRIVAL,NaN,NaN,NaN,532.0,2017-01-17 06:25:00,20170115,...,1.0,3.0,15.0,1.0,1.0,2.0,1.0,0.0,1.0,0.409264


In [3]:
data.columns

Index(['order_id', 'order_date', 'logistic_order_id', 'action', 'facility_id',
       'facility_type', 'city_id', 'logistic_company_id', 'timestamp', 'day',
       'item_det_info', 'pay_timestamp', 'buyer_id', 'promise_speed',
       'if_cainiao', 'merchant_id', 'Logistics_review_score', 'min', 'max',
       'action_time', 'day_count', 'avg_action_time', 'action_count',
       'facility_count', 'week', 'day_values', 'day_of_week', 'recieve_count',
       'arrive_count', 'depart_count', 'scan_count', 'consign_count',
       'post_action_time'],
      dtype='object')

# day of week

In [4]:
df = data[['order_id','order_date','action','action_time','timestamp','logistic_company_id','day_count','avg_action_time','Logistics_review_score']]
df.head(10)

,order_id,order_date,action,action_time,timestamp,logistic_company_id,day_count,avg_action_time,Logistics_review_score
0,100534527,20170115,CONSIGN,0.127702,2017-01-16 08:49:00,532.0,4,0.307430,5.0
1,100534527,20170115,SIGNED,1.000000,2017-01-19 16:51:00,532.0,4,0.307430,5.0
2,100534527,20170115,GOT,0.227793,2017-01-16 18:00:00,532.0,4,0.307430,5.0
3,100534527,20170115,DEPARTURE,0.368392,2017-01-17 06:54:00,532.0,4,0.307430,5.0
4,100534527,20170115,ARRIVAL,0.363124,2017-01-17 06:25:00,532.0,4,0.307430,5.0
5,100534527,20170115,ARRIVAL,0.450136,2017-01-17 14:24:00,532.0,4,0.307430,5.0
6,100534527,20170115,order,0.000000,2017-01-15 21:06:00,NaN,4,0.307430,5.0
7,60175737,20170109,SIGNED,1.000000,2017-01-11 14:33:00,247.0,2,0.633054,5.0
8,60175737,20170109,ARRIVAL,0.862603,2017-01-11 07:18:00,247.0,2,0.633054,5.0
9,60175737,20170109,SENT_SCAN,0.965572,2017-01-11 12:44:00,247.0,2,0.633054,5.0


In [5]:
# start day of week
# Monday: 0
df_start = df.query('action == "CONSIGN"')
df_start['start_day'] = pd.to_datetime(df_start.order_date, format='%Y%m%d').dt.dayofweek
df_start.head()

,order_id,order_date,action,action_time,timestamp,logistic_company_id,day_count,avg_action_time,Logistics_review_score,start_day
0,100534527,20170115,CONSIGN,0.127702,2017-01-16 08:49:00,532.0,4,0.307430,5.0,6
11,60175737,20170109,CONSIGN,0.078016,2017-01-09 13:54:00,247.0,2,0.633054,5.0,0
22,34518067,20170122,CONSIGN,0.087702,2017-01-22 14:17:00,273.0,1,0.499853,5.0,6
29,111264400,20170102,CONSIGN,0.192362,2017-01-02 11:31:00,247.0,2,0.428563,5.0,0
37,9815812,20170103,CONSIGN,0.030843,2017-01-03 19:29:00,273.0,2,0.293557,5.0,1


In [6]:
df_start.start_day.value_counts()

6    4330
1    3928
0    3912
2    3644
3    3590
4    3255
5    3017
Name: start_day, dtype: int64

# Regression on avg_action_time(x) Logistics_review_score(y)
## dummy variable: day of week

In [7]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm

# Choose the features to be used
cols = ['start_day']
X_train = df_start[cols]
y_train = df_start['avg_action_time']

X_train = pd.get_dummies(X_train, columns=['start_day'])
# X_train = sm.add_constant(X_train)

# fit the data to the model
model = sm.OLS(y_train, X_train).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        avg_action_time   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.417
Date:                Sat, 20 Nov 2021   Prob (F-statistic):            0.00226
Time:                        09:58:25   Log-Likelihood:                 16119.
No. Observations:               25676   AIC:                        -3.222e+04
Df Residuals:                   25669   BIC:                        -3.217e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
start_day_0     0.4968      0.002    240.536      

In [8]:
X_train

,start_day_0,start_day_1,start_day_2,start_day_3,start_day_4,start_day_5,start_day_6
0,0,0,0,0,0,0,1
11,1,0,0,0,0,0,0
22,0,0,0,0,0,0,1
29,1,0,0,0,0,0,0
37,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
212307,0,0,1,0,0,0,0
212312,0,0,0,0,1,0,0
212323,0,0,0,0,0,0,1
212330,1,0,0,0,0,0,0


In [9]:
y_train

0         0.307430
11        0.633054
22        0.499853
29        0.428563
37        0.293557
            ...   
212307    0.574117
212312    0.532375
212323    0.445255
212330    0.506725
212337    0.502098
Name: avg_action_time, Length: 25676, dtype: float64

In [10]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm

# Choose the features to be used
cols = ['start_day', 'avg_action_time']
X_train = df_start[cols]
y_train = df_start['Logistics_review_score']

X_train = pd.get_dummies(X_train, columns=['start_day'])
# X_train = sm.add_constant(X_train)

# fit the data to the model
model = sm.OLS(y_train, X_train).fit()

print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Logistics_review_score   R-squared:                       0.001
Model:                                OLS   Adj. R-squared:                  0.001
Method:                     Least Squares   F-statistic:                     4.872
Date:                    Sat, 20 Nov 2021   Prob (F-statistic):           1.66e-05
Time:                            09:58:30   Log-Likelihood:                -23846.
No. Observations:                   25676   AIC:                         4.771e+04
Df Residuals:                       25668   BIC:                         4.777e+04
Df Model:                               7                                         
Covariance Type:                nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
av

In [11]:
X_train

,avg_action_time,start_day_0,start_day_1,start_day_2,start_day_3,start_day_4,start_day_5,start_day_6
0,0.307430,0,0,0,0,0,0,1
11,0.633054,1,0,0,0,0,0,0
22,0.499853,0,0,0,0,0,0,1
29,0.428563,1,0,0,0,0,0,0
37,0.293557,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
212307,0.574117,0,0,1,0,0,0,0
212312,0.532375,0,0,0,0,1,0,0
212323,0.445255,0,0,0,0,0,0,1
212330,0.506725,1,0,0,0,0,0,0


## dummy variable: day_count

In [13]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm

# Choose the features to be used
cols = ['day_count', 'avg_action_time']
X_train = df_start[cols]
y_train = df_start['Logistics_review_score']

X_train = pd.get_dummies(X_train, columns=['day_count'])
# X_train = sm.add_constant(X_train)

# fit the data to the model
model = sm.OLS(y_train, X_train).fit()

print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Logistics_review_score   R-squared:                       0.015
Model:                                OLS   Adj. R-squared:                  0.014
Method:                     Least Squares   F-statistic:                     34.43
Date:                    Sat, 20 Nov 2021   Prob (F-statistic):           7.53e-74
Time:                            09:58:41   Log-Likelihood:                -23675.
No. Observations:                   25676   AIC:                         4.737e+04
Df Residuals:                       25664   BIC:                         4.747e+04
Df Model:                              11                                         
Covariance Type:                nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
av

## dummy variable: day of week, day_count

In [14]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm

# Choose the features to be used
cols = ['start_day', 'day_count', 'avg_action_time']
X_train = df_start[cols]
y_train = df_start['Logistics_review_score']

X_train = pd.get_dummies(X_train, columns=['start_day', 'day_count'])
# X_train = sm.add_constant(X_train)

# fit the data to the model
model = sm.OLS(y_train, X_train).fit()

print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Logistics_review_score   R-squared:                       0.015
Model:                                OLS   Adj. R-squared:                  0.014
Method:                     Least Squares   F-statistic:                     22.53
Date:                    Sat, 20 Nov 2021   Prob (F-statistic):           2.49e-70
Time:                            09:58:46   Log-Likelihood:                -23672.
No. Observations:                   25676   AIC:                         4.738e+04
Df Residuals:                       25658   BIC:                         4.753e+04
Df Model:                              17                                         
Covariance Type:                nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
av

## Consign action time

Warehouse-to-shipper consignment is always the first action to follow the customer’s order, so delaying this consignment delays all subsequent actions. But this consignment happens before the shipment, so the consignment time should not directly affect the shipment quality (conditional on the final delivery time).

In [15]:
# Import the library that contains all the functions/modules related to the regression model
import statsmodels.api as sm

# Choose the features to be used
cols = ['action_time', 'avg_action_time']
X_train = df_start[cols]
y_train = df_start['Logistics_review_score']

# X_train = pd.get_dummies(X_train, columns=['start_day'])
X_train = sm.add_constant(X_train)

# fit the data to the model
model = sm.OLS(y_train, X_train).fit()

print(model.summary())

                              OLS Regression Results                              
Dep. Variable:     Logistics_review_score   R-squared:                       0.001
Model:                                OLS   Adj. R-squared:                  0.001
Method:                     Least Squares   F-statistic:                     17.19
Date:                    Sat, 20 Nov 2021   Prob (F-statistic):           3.46e-08
Time:                            09:58:49   Log-Likelihood:                -23845.
No. Observations:                   25676   AIC:                         4.770e+04
Df Residuals:                       25673   BIC:                         4.772e+04
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
co